In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
# 多层感知机
net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10),
)
X = torch.rand(2, 20)
net(X)

tensor([[ 0.0193,  0.1957,  0.0058,  0.1195,  0.0642, -0.2791,  0.1832,  0.0410,
          0.1202,  0.0835],
        [ 0.1011,  0.3723, -0.1108,  0.3213,  0.1109, -0.0227,  0.1090, -0.0565,
          0.1628,  0.1060]], grad_fn=<AddmmBackward0>)

In [3]:
# 自定义块
class MLP(nn.Module):
    """ 多层感知机实现 """
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, x):
        """ 前向传播 """
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
print(net(X))
print(net(X))

tensor([[-0.2799, -0.1270, -0.0358,  0.1005, -0.0684,  0.0355, -0.1673,  0.3204,
         -0.0537, -0.0223],
        [-0.2764, -0.0953,  0.0438,  0.1827, -0.0887,  0.0115, -0.1569,  0.4264,
         -0.1687,  0.0591]], grad_fn=<AddmmBackward0>)
tensor([[-0.2799, -0.1270, -0.0358,  0.1005, -0.0684,  0.0355, -0.1673,  0.3204,
         -0.0537, -0.0223],
        [-0.2764, -0.0953,  0.0438,  0.1827, -0.0887,  0.0115, -0.1569,  0.4264,
         -0.1687,  0.0591]], grad_fn=<AddmmBackward0>)


In [5]:
class MySequential(nn.Module):
    """ 顺序块 """
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # _modules的主要优点是： 在模块的参数初始化过程中， 
            # 系统知道在_modules字典中查找需要初始化参数的子块。
            self._modules[str(idx)] = module  ## _module的类型是OrderedDict
    
    def forward(self, X):
        """ 前向传播 """
        for block in self._modules.values():
            X = block(X)
        return X

In [6]:
net = MySequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10),
)
net(X)

tensor([[-0.1007,  0.0519, -0.1196, -0.1756,  0.0676,  0.0154,  0.2787, -0.0085,
         -0.0165, -0.0911],
        [-0.0992,  0.0866,  0.0042, -0.1856,  0.0131,  0.1237,  0.2857, -0.1415,
         -0.0242,  0.0115]], grad_fn=<AddmmBackward0>)

In [7]:
class FixedHiddenMLP(nn.Module):
    """ 固定常量 """
    def __init__(self):
        super().__init__()
        # 固定随机权重参数，其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        """ 前向传播 """
        X = self.linear(X)
        # 使用创建的常量参数以及relu和num函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层,当于两个全连接层共享参数
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [8]:
net = FixedHiddenMLP()
net(X)

tensor(0.1486, grad_fn=<SumBackward0>)

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0528, grad_fn=<SumBackward0>)

In [10]:
# 参数管理
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.3891],
        [0.4553]], grad_fn=<AddmmBackward0>)

In [11]:
# 访问参数
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2880,  0.1276, -0.1241,  0.2443,  0.2370,  0.3315, -0.2618, -0.2396]])), ('bias', tensor([0.1385]))])


In [12]:
# 每个参数都表示为参数类的一个实例
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1385], requires_grad=True)
tensor([0.1385])


In [13]:
# 数是复合的对象，包含值、梯度和额外信息
net[2].weight.grad == None

True

In [14]:
# 一次性访问所有的参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])
print(net.state_dict()['2.bias'].data)

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))
tensor([0.1385])


In [15]:
# 从嵌套块收集参数
def block1():
    net = nn.Sequential(
        nn.Linear(4, 8),
        nn.ReLU(),
        nn.Linear(8, 4),
        nn.ReLU()
    )
    return net
    
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.1317],
        [-0.1318]], grad_fn=<AddmmBackward0>)

In [16]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [17]:
rgnet[0][1][0].bias.data

tensor([ 0.4213, -0.4938, -0.4728, -0.3758,  0.4617,  0.3971, -0.2798,  0.2159])

In [18]:
# 参数初始化
# 默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
    
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0007,  0.0054,  0.0011, -0.0019]), tensor(0.))

In [19]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [20]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.0170,  0.2385, -0.5455,  0.2954])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [21]:
def my_init(m):
    """ 自定义初始化 """
    if type(m) == nn.Linear:
        print("init", *[(name, param.shape) 
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
        
net.apply(my_init)
net[0].weight[:2]

init weight torch.Size([8, 4])
init weight torch.Size([1, 8])


tensor([[-0.0000, -8.4307, -0.0000, -7.0876],
        [-0.0000, -0.0000, -6.1127, -9.0216]], grad_fn=<SliceBackward0>)

In [22]:
# 直接赋值
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -7.4307,  1.0000, -6.0876])

In [23]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


当参数绑定时，梯度会发生什么情况？ 答案是由于模型参数包含梯度，因此在反向传播期间第二个隐藏层 （即第三个神经网络层）和第三个隐藏层（即第五个神经网络层）的梯度会加在一起。

延后初始化
#深度学习框架无法判断网络的输入维度是什么。 这里的诀窍是框架的延后初始化（defers initialization）， 即直到数据第一次通过模型传递时，框架才会动态地推断出每个层的大小。

In [24]:
del net

In [25]:
import torch
from torch import nn
from d2l import torch as d2l

In [26]:
net = nn.Sequential(
    nn.LazyLinear(256),
    nn.ReLU(),
    nn.LazyLinear(10),
)
print(net[0].weight)

<UninitializedParameter>


D:\Anaconda\envs\wz_py_3_8\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [27]:
X = torch.rand(2, 20)
y = net(X)

print(net[0].weight.shape)
print(net[2].weight.shape)
print(y.shape)

torch.Size([256, 20])
torch.Size([10, 256])
torch.Size([2, 10])


In [28]:
@d2l.add_to_class(d2l.Module)  #@save
def apply_init(self, inputs, init=None):
    self.forward(*inputs)
    if init is not None:
        self.net.apply(init)

In [29]:
# 自定义层
class CenteredLayer(nn.Module):
    """ 构建一个没有参数的层 """
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X - X.mean()
    
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [30]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [31]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(3.2596e-09, grad_fn=<MeanBackward0>)

下面我们继续定义具有参数的层， 这些参数可以通过训练进行调整。 我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。 比如管理访问、初始化、共享、保存和加载模型参数。 这样做的好处之一是：我们不需要为每个自定义层编写自定义的序列化程序。

In [32]:
class MyLinear(nn.Module):
    """ 带参数的层 """
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        print(self.weight.shape)
        self.bias = nn.Parameter(torch.randn(units,))
        print(self.bias.shape)
        
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [33]:
linear = MyLinear(5, 3)
linear.weight

torch.Size([5, 3])
torch.Size([3])


Parameter containing:
tensor([[ 1.3298,  1.1064,  2.2462],
        [-0.1072,  1.2640, -0.5304],
        [-1.8276,  0.1784,  0.3739],
        [ 0.4672, -0.6441, -0.5024],
        [ 0.8355,  0.9857, -0.6726]], requires_grad=True)

In [34]:
linear(torch.rand(2, 5))

tensor([[0.0000, 2.5648, 0.0000],
        [0.3851, 2.8038, 0.0000]])

In [35]:
# 读写文件，加载和存储权重向量和整个模型
x = torch.arange(4)
torch.save(x, './data/x-file')

In [36]:
x2 = torch.load('./data/x-file')
print(x2)

tensor([0, 1, 2, 3])


In [37]:
y = torch.zeros(4)
torch.save([x, y],'./data/x-file')
x2, y2 = torch.load('./data/x-file')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [38]:
mydict = {'x': x, 'y': y}
torch.save(mydict, './data/mydict')
mydict2 = torch.load('./data/mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [39]:
# 保存模型参数
class MLP(nn.Module):
    """ 多层感知机 """
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))
    
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [40]:
torch.save(net.state_dict(), './data/mlp.params')

In [41]:
clone = MLP()
clone.load_state_dict(torch.load('./data/mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [42]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [43]:
!nvidia-smi

Tue Aug 20 15:34:25 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 556.12                 Driver Version: 556.12         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   55C    P8             15W /  115W |     182MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [44]:
# 查询可用GPU数量
torch.cuda.device_count()

1

In [45]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [46]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [47]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [48]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [49]:
net(X)

tensor([[-0.2260],
        [-0.2260]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [50]:
net[0].weight.data.device

device(type='cuda', index=0)